In [1]:
import os
import pathlib

import numpy as np
import pandas as pd

import shapely.geometry
import rasterio
import rasterio.plot
import rasterio.mask

import rasterio.features
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import cmasher

from util_data_exploration import get_tile_images, reconvert_np_to_rasterio_dataset

## Contents of notebook
- collecting all relevant scenario folders for processing
- processing terrain maps, flood maps, and Rotterdam district areas

This notebook specifically processes flood scenario data from LIWO, filters it with the terrain (DSM) data from AHN, and separates the flood data per district (wijk) in Rotterdam.

## Usage notes
- LIWO flood data should be placed in the data/flood_breaks directory, as separate directories per scenario. This script should automatically detect the scenarios.
- The AHN terrain data (Digital Surface Mapping) data file is pre-processed and present in data/rdam_topo_bool_map.tif . The original raw data from AHN consists of multiple tiles and needed to be combined together, which was done in a different script (TODO: explain that script)
- the Rotterdam district vector data is pre-processed and present in data/wijk_geometries.shp. Note that this script takes only a subset of the data, since industry-only districts and the port district of R'dam is omitted. Note that the district labels across different datasets can be different (some explicitly list all the industry districts, others just lump the industry districts as a generic 'bedrijventerrein' catch-all category)

In [2]:
# Search for flood scenario directories in data/flood_breaks
base_path = '../data/flood_breaks'

# find valid folders
target_ending = '.tif'  # look for tiff files only
scenario_paths = []  # directory paths to the scenario files
scenario_names = []  # names of the scenarios
for path in os.listdir(f'{base_path}'):
    if '.zip' not in path:  ## exclude zip files, look for folders
        for item in os.listdir(f'{base_path}/{path}'):
            if target_ending in item:  # if tiff file
                scenario_paths.append(f'{base_path}/{path}/{item}')  # save relative path
                scenario_names.append(path.lower())  # the scenario names are on the directory, not the tif file itself.

In [3]:
# extract and filter Rotterdam district VECTOR data for filtering later on
wijken = gpd.read_file('../data/wijk_geometries.shp')
wijken.WK_NAAM = wijken.WK_NAAM.str.replace(' ',
                                            '-').str.lower()  # switch to lower case and replace all spaces with hyphens for consistency with other datasets
wijken.set_index('WK_NAAM', inplace=True)  # set district name as index

# filter for districts of interest
wijken_of_interest = ['Rotterdam Centrum', 'Delfshaven', 'Overschie', 'Noord', 'Hillegersberg-Schiebroek',
                      'Kralingen-Crooswijk', 'Prins Alexander', 'Feijenoord', 'IJsselmonde', 'Charlois', 'Pernis',
                      'Hoogvliet', 'Hoek van Holland', 'Rozenburg']  # derived from households and companies dataframe
wijken_of_interest = [name.replace(' ', '-').lower() for name in wijken_of_interest]
wijken = wijken.loc[wijken_of_interest, :]

# only use districts that are included
wijks_used = pd.read_pickle('../data_model_inputs/households_brackets_per_wijk_city_only.pickletable')



## Process Rotterdam terrain data for filtering
The following block processes the terrain data from AHN. The pre-processed data is a rasterio dataset object containing 0 for water and 1 for land, and will be used to filter out inland bodies of water in the flood dataset (some water bodies are marked as flooded, even though there won't be any buildings on them).

The procedure uses a hacky method based on <https://gis.stackexchange.com/questions/363183/crop-two-rasters-to-common-extent-where-pixelvalues-are-not-nodata>, converting the terrain (raster) to vector to be used as a mask for the flood map.

In [4]:
# Read rotterdam land bool map, generated from map_explore.ipynb
terrain_bool_ds = rasterio.open('../data/rdam_topo_bool_map.tif')
terrain_bool_map = terrain_bool_ds.read(1)  # derived from map_explore.ipynb
# in the map 0 = water, 1 = land
# convert terrain map (raster) to vector map for masking.

knockout_meta = terrain_bool_ds  # with metadata
knockout = terrain_bool_map
geoms_water = []
geoms_land = []

# convert the raster to vector features, for masking out inland water bodies later on
for geometry, raster_value in rasterio.features.shapes(knockout, transform=knockout_meta.transform):
    if raster_value == 0:  # 0 = water, 1 = land
        geoms_water.append(geometry)
    else:
        geoms_land.append(geometry)


## Processing flood data, with filtering/masking from the terrain map and district boundaries.
This section processes the flood data with the terrain maps (to ignore inland water) and district geometry. The flood data will be separated according to the districts, and saved into a pandas dataframe (currently 'data_model_inputs/flood_scenarios_v2.pickletable') for further usage. An entry in the dataframe includes the distribution of land 'tiles'.



In [5]:
# NEW RETROFIT FOR THESIS PLOTTING

# use generated vectors as mask. Should be less sensitive to differential spatial resolution than older method in map_explore.ipynb

# TODO: list out assumptions in this datacleaning

temp_dict = {}  # for temporary storage, for generation of pandas dataframe
flooded_pct_threshold = .01   # percentage of flooded landmass to be considered significant (1%)
force_equalise = True  # bool argument to force non-data (but still landmass) to 0 (TODO: explain)

for idx, path in enumerate(scenario_paths):
    with rasterio.open(path) as input_flood_ds:  # open the flood scenario in 'safe' manner
        input_flood_scenario = scenario_names[idx]  # get scenario name
        # first mask out inland water bodies to minimise invalid flood values
        out_flood_img, out_transform = rasterio.mask.mask(dataset=input_flood_ds,
                                                          shapes=geoms_land,
                                                          crop=True,
                                                          nodata=np.nan,
                                                          pad=True)
        # use the first band (count=1) for further cropping, it contains all flood data. Other bands contain bins of flood depths
        out_flood_ds = reconvert_np_to_rasterio_dataset(raster=out_flood_img[0, :, :],
                                                        transform=out_transform,
                                                        count=1)  # only use the 1st band

        # crop out the flood map with the wijk borders, only for the wijken of interest
        cropped, transform = rasterio.mask.mask(dataset=out_flood_ds,
                                                shapes=wijken.loc[wijks_used.index,'geometry'],
                                                crop=True,
                                                nodata=np.nan)

        # filter out very small flooding that can be considered negligible (for plotting)
        cropped = np.where(cropped < flooded_pct_threshold, 0., cropped)
        temp_dict[input_flood_scenario] = (cropped, transform)  #  save as tuple for plotting

        # cut out the flood submaps with the R'dam district shapes
        # filter out districts that are not affected
        # caveat: dependent on which wijken are collected for this study (eg. the business-specific areas are not included)
        # for wijk in wijken.index:  # numeric index
        #     geo = wijken.geometry[wijk]
        #
        #     # cut out flood values belonging to the specific wijk
        #     try:
        #         submap, transform = rasterio.mask.mask(dataset=out_flood_ds,
        #                                                shapes=[geo],
        #                                                crop=True,
        #                                                nodata=np.nan)
        #         # process to see if map contains any valid flood values
        #         submap_depths = submap[submap > 0.]  # 1d array of parcels higher than 0
        #         if submap_depths.size > 0:  # if there are areas higher than 0
        #
        #             # if non-data exists on the cropped submap, force them to 0. (A)
        #             if force_equalise:
        #                 submap = np.where(submap < 0., 0., submap)
        #
        #             # gets the distribution of land parcels. Used for sampling buildable land
        #             flatten = submap[submap >= 0.]
        #             if submap_depths.size / flatten.size >= flooded_pct_threshold: # if flooded proportion is higher than threshold
        #
        #                 # save to dict for DataFrame generation
        #                 temp_dict[f'{input_flood_scenario},{wijk}'] = {'scenario': input_flood_scenario,
        #                                                                'wijk': wijk,
        #                                                                'submap': submap,
        #                                                                'transform': transform,
        #                                                                'flatten': flatten}
        #
        #     except ValueError:  # occurs if the map geometry doesn't overlap flood area
        #         pass  # no appending to flood_submaps
        #         # flood_submaps[name] = None # could also ignore

In [11]:
# plot the flood dataset with the map borders

colour_choice = mpl.colormaps['GnBu']
modded = ListedColormap(colour_choice(np.linspace(0.2, 1.0, 205)))
test_mode = False
if test_mode:
    plt.ion()  # plot in notebook
else:
    plt.ioff() # only output as file

for fld_scenario, fld_cropped in temp_dict.items():
    fld_map, fld_transform = fld_cropped

    fig, ax = plt.subplots(figsize=(10,8))

    # plot cropped floodmap with colour
    rasterio.plot.show(fld_map, transform=fld_transform, ax=ax, cmap=cmasher.arctic_r, vmin=0, vmax=3.)
    # plot borders of wijken of interest
    wijken.loc[wijks_used.index].boundary.plot(ax=ax, edgecolor='grey')
    brightness = 0.8  # for RGB setting of grey
    fig.set_facecolor((brightness, brightness,brightness))  # create grey background
    ax.axis('off')  # turn off axes
    plt.tight_layout()  # calculate plot spacing for output
    print(f'{fld_scenario} created')

    if test_mode:
        break
    else:
        # save fig and clear for next iteration
        fig.savefig(f'../data_model_outputs/fld_plots/{fld_scenario}.png', dpi=500)
        fig.clf()
        ax.cla()



boerengatsluis_1e6 created
capelle_west_nijverheidstraat_1e5 created
maashaven_4e5 created
maashaven_4e5kv2 created
nieuwe_maas_4e5.km41.2 created
parksluizen_1e4 created
parksluizen_1e6 created


<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

In [9]:
if temp_dict:  # if dict is not empty

    flood_submaps_df = pd.DataFrame.from_dict(temp_dict, orient='index').astype(
        {'scenario': 'category',
         'wijk': 'category'})
    flood_submaps_df['flatten_size'] = [v.size for v in flood_submaps_df.flatten]
    flood_submaps_df['submap_size'] = [v.size for v in flood_submaps_df.submap]

    # clean and truncate floodmaps since the flood-data borders are inconsistent

    # save dataframe to disk (uncomment when necessary)
    # flood_submaps_df.to_pickle('../data_model_inputs/flood_scenarios_v2.pickletable')

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'scenario' not found in columns."